# Fatoração $PA= LU$

In [1]:
def elimina_gauss( A ):
    
    n = A.shape[ 0 ]
    L = np.eye( n )
    U = A.copy()
    
    retval = [ [ L.copy(), U.copy() ] ]
    
    for i in range( n ):
        for k in range( i + 1, n ):
            
            alpha = U[ k, i ] / U[ i, i ]
            
            for j in range( i, n ):
                U[ k, j ] = U[ k, j ] - alpha * U[ i, j ]
                
            L[ k, i ] = alpha
            
            retval.append( [ L.copy(), U.copy() ] )
            
    return retval

def mostra_matriz(
    A_list,
    b = None,
    format_str = [ '%f' ],
    l_delim = '(', r_delim = ')',
    spacer = [ '' ],
    thres = [ None ],
    destaque_red = [],
    destaque_blue = [],
    destaque_green = [],
    quiet = False,
    replace = [ '' ],
    replace_other = None
):
    out_latex = '$$\n'
    count = 0
    for A in A_list:
        
        if thres[ count % len( thres ) ] is None:
            thres[ count % len( thres ) ] = np.full( A.shape, -1e-5 )
        
        ( m, n ) = A.shape

        out_latex += '   \\left' + l_delim + '\n      \\begin{array}{' + ( n * 'r' ) + '}\n'
        for i in range( m ):
            out_latex += '         '
            for j in range( n ):
                if abs( A[ i, j ] ) >= thres[ count % len( thres ) ][ i, j ]:
                    if not ( replace_other is None ):
                        out_latex += replace_other[ count % len( replace_other ) ] + '&'
                    elif ( count, i, j ) in destaque_green:
                        out_latex += '\\color{green}{'
                        out_latex += ( format_str[ count % len( format_str ) ] + '}&' ) % ( A[ i, j ], )
                    elif ( count, i, j ) in destaque_red:
                        out_latex += '\\color{red}{'
                        out_latex += ( format_str[ count % len( format_str ) ] + '}&' ) % ( A[ i, j ], )
                    elif ( count, i, j ) in destaque_blue:
                        out_latex += '\\color{blue}{'
                        out_latex += ( format_str[ count % len( format_str ) ] + '}&' ) % ( A[ i, j ], )
                    else:
                        out_latex += ( format_str[ count % len( format_str ) ] + '&' ) % ( A[ i, j ], )
                else:
                    out_latex += replace[ count % len( replace ) ] + '&'
            out_latex = out_latex[ : -1 ]
            if i < m - 1:
                out_latex += '\\\\'
            out_latex += '\n'

        out_latex += '      \\end{array}\n   \\right' + r_delim + spacer[ count % len( spacer ) ]
        count = count + 1
    
    out_latex += '\n$$'
    
    if not quiet:
        dp.display_latex( dp.Latex( out_latex ) )
    return out_latex

In [2]:
import numpy as np
import IPython.display as dp

In [3]:
A = np.array( [ [ 1e-20, 1 ], [ 1, 1 ] ] )
_ = mostra_matriz(
    [
        A
    ],
    thres = [ np.full( A.shape, -1e-5 ) ],
    format_str = [ '%e' ]
)

In [4]:
res = elimina_gauss( A )

_ = mostra_matriz(
    [
        res[ -1 ][ 0 ] @ res[ -1 ][ 1 ]
    ],
    thres = [ np.full( A.shape, -1e-5 ) ],
    format_str = [ '%e' ]
)

Podemos pensar a eliminação gaussiana como um processo de inclusão de zeros da seguinte maneira:

$$\large
L_{n - 1}\cdots L_2L_1A = U \Rightarrow L^{-1}A = U \Rightarrow A = LU
$$

In [5]:
n = 10
A = np.eye( 10 )
j = 3
for i in range( j + 1, n ):
    A[ i, j ] = '-1'
_ = mostra_matriz(
    [
        A
    ],
    thres = [ np.full( A.shape, 1e-5 ) ],
    format_str = [ '%.f' ]
)

In [6]:
def LU_pp( A ):
    
    n = A.shape[ 0 ]
    p = list( range( n ) )
    
    for i in range( n ):
        
        I = np.argmax( np.abs( A[ i :, i ] ) ) + i
    
        tmp = A[ i, : ].copy()
        A[ i, : ] = A[ I, : ]
        A[ I, : ] = tmp
        
        tmp = p[ i ]
        p[ i ] = p[ I ]
        p[ I ] = tmp
        
        alpha = A[ i + 1 :, [ i ] ] / A[ i, i ]
        A[ i + 1 :, i : ] = A[ i + 1 :, i : ] - alpha * A[ i, i : ]
        
        A[ i + 1 :, i ] = alpha.flatten()

    return ( A, p )
        
def LU_separa( A ):
    
    U = np.triu( A )
    L = np.eye( A.shape[ 0 ] )
    L = L + np.tril( A, -1 )
    
    return ( L, U )

def triu_solve( U, b ):
    
    n = U.shape[ 0 ]    
    x = np.empty( ( b.shape[ 0 ], ) )
    
    for i in range( n - 1, -1, -1 ):
        
        x[ i ] = ( b[ i ] - np.sum( U[ i, i + 1 : n ] * x[ i + 1 : n ] ) ) / U[ i, i ]
        
    return x

def tril_solve( L, b ):
    
    n = L.shape[ 0 ]    
    x = np.empty( b.shape )
    
    for i in range( n ):
        
        x[ i ] = b[ i ] - np.sum( L[ i, : i ] * x[ : i ] )
        
    return x

def inversa_pp( A ):
    
    A_copy = A.copy()
    ( A_copy, p ) = LU_pp( A_copy )
    
    n = A.shape[ 0 ]
    
    inv = np.empty( ( n, n ) )
    
    for j in range( n ):
        
        e = np.zeros( ( n, ) )
        e[ j ] = 1.0
        
        y = tril_solve( A_copy, e[ p ] )
        inv[ :, j ] = triu_solve( A_copy, y )
        
    return inv

def LU( A ):
    
    n = A.shape[ 0 ]
    
    for i in range( n ):
        alpha = A[ i + 1 : n, [ i ] ] / A[ i, i ]
        A[ i + 1 : n, i : n ] = A[ i + 1 : n, i : n ] - alpha * A[ i, i : n ]
        
        A[ i + 1 : n, i ] = alpha.flatten()

def inversa( A ):
    
    A_copy = A.copy()
    LU( A_copy )
    
    n = A.shape[ 0 ]
    
    inv = np.empty( ( n, n ) )
    
    for j in range( n ):
        
        e = np.zeros( ( n, ) )
        e[ j ] = 1.0
        
        y = tril_solve( A_copy, e )
        inv[ :, j ] = triu_solve( A_copy, y )
        
    return inv

In [7]:
n = 400
A = np.random.normal( size = ( n, n ) )

In [8]:
%timeit A_inv = inversa( A )

1.93 s ± 39.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
A_inv = inversa( A )

In [10]:
%timeit A_inv_pp = inversa_pp( A )

1.93 s ± 31.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
A_inv_pp = inversa_pp( A )

In [12]:
print( np.max( np.abs( A_inv @ A - np.eye( n ) ) ) )
print( np.max( np.abs( A_inv_pp @ A - np.eye( n ) ) ) )

9.293537051036083e-10
8.615330671091215e-14


# Sistemas de equações não-lineares

Passaremos a abordar a solução de sistemas de equações não-lineares do tipo

$$\large\def\vect{\boldsymbol}
\vect F( \vect x ) = \vect 0,
$$
onde $\vect F: \mathbb R^n \to \mathbb R^n$.

Ou seja,
$$\large
\vect F( \vect x ) = \begin{pmatrix}F_1( \vect x )\\F_2( \vect x )\\\vdots\\F_n( \vect x )\end{pmatrix}
$$

onde $F_i : \mathbb R^n \to \mathbb R$.

Desta forma, nosso sistema de equações não-lineares é equivalente a resolver
Ou seja,
$$\large
\begin{split}
    F_1( \vect x ) &{}= 0\\
    F_2( \vect x ) &{}= 0\\
    &{}\vdots\\
    F_n( \vect x ) &{}= 0
\end{split}
$$